In [1]:
import math
import nltk
from nltk import word_tokenize,sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/rajarshidas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# PREPROCESSING

In [2]:
text=""
for i in range (1,7):
    filename='Harry_Potter_Text/'+'Book'+str(i)+'.txt'
    with open(filename) as f:
        Lines = f.readlines()
        # book=""
        for lin in Lines:
            if not (len(lin)>5 and lin[:6]=='Page |'):
                if lin != 'Harry Potter and the Goblet of Fire - J.K. Rowling \n':
                    text+=lin.lower()
x=sent_tokenize(text)
sentences=[]
sentencewise_tokens=[]
for l in x:
    li=''.join(l.splitlines())
    sentences.append(li)
    sentencewise_tokens.append(word_tokenize(li))
tokens=word_tokenize(text)



In [3]:
len(sentences)

51670

In [4]:
# sentences

In [5]:
len(tokens)

1155432

In [6]:
len(sentencewise_tokens)

51670

In [7]:
final_tokens=[]
for token in tokens:
    if not (len(token)==1 and not token.isalnum()):
        final_tokens.append(token.strip('.'))
tokens=final_tokens
#REMOVE PUNCTUATION

In [8]:
len(tokens)

920732

In [9]:
vocab=list(set(tokens))

In [10]:
print(len(vocab))

21079


In [45]:
train_book=' '.join(tokens)

# N-GRAM MODEL

In [11]:
#value of m in problem statement/ range of n in n-gram
n_limit=5

In [12]:
ngrams=[]
for i in range (n_limit):
    d={}
    ngrams.append(d)
for n in range (1,n_limit+1):
    for i in range(len(tokens)):
        if i+n>len(tokens):
            break
        lis=[]
        for j in range(i,i+n):
            lis.append(tokens[j])
        lis=tuple(lis)
        if lis not in ngrams[n-1]:
            ngrams[n-1][lis] = 1
        else : 
            ngrams[n-1][lis] += 1
    ngrams[n-1]=dict(sorted(ngrams[n-1].items(), key=lambda item: -item[1]))
    print(n,len(ngrams[n-1]))




1 21079
2 277884
3 642415
4 835723
5 896438


In [13]:
# ngrams[4]

# SENTENCE GENERATION

In [14]:
def get_n(n,a,line,word_limit):
    if word_limit==0:
        print(line)
        return line
    sz=len(a)
    if sz==0 or n==1:
        word=next(iter(ngrams[0]))[0]#first key
        a.append(word)
        line=line+' '+word
        # print(line)
        get_n(n,a,line,word_limit-1)
    elif sz<n-1:
        found = False
        for key in ngrams[sz]:
            lis=list(key)
            last_word=lis[-1]
            lis=lis[:-1]
            if lis==a:
                line=line+' '+last_word
                # a=a[1:]
                a.append(last_word)
                found=True
                break
        if found :
            # print(line)
            get_n(n,a,line,word_limit-1)
        else : 
            print('Cant find')
    elif sz==n-1:
        found = False
        for key in ngrams[n-1]:
            lis=list(key)
            last_word=lis[-1]
            lis=lis[:-1]
            if lis==a:
                line=line+' '+last_word
                a=a[1:]
                a.append(last_word)
                found=True
                break
        if found :
            # print(line)
            get_n(n,a,line,word_limit-1)
        else : 
            print('Cant find')
        




In [15]:
def generate_sentence(n,word_limit):
    prev_tokens=[]
    sentence=""
    line=get_n(n,prev_tokens,sentence,word_limit)  


In [16]:
generate_sentence(5,10)

 the door and turned the key fumbling in their panic


In [17]:
generate_sentence(5,100)

 the door and turned the key fumbling in their panic harry pulled the door open and they ran inside hermione granger was shrinking against the wall opposite looking as if she was about to faint the troll was advancing on her knocking the sinks off the walls as it went confuse it harry said desperately to ron and seizing a tap he threw it as hard as he could into the kitchen harry hurried into the living room in time to catch the last report on the evening news and finally bird-watchers everywhere have reported that the nation s owls


# DEV SET

In [180]:
text=""
filename='Harry_Potter_Text/'+'Book7.txt'
with open(filename) as f:
    Lines = f.readlines()
    # book=""
    for lin in Lines:
        if not (len(lin)>5 and lin[:6]=='Page |'):
            if lin != 'Harry Potter and the Goblet of Fire - J.K. Rowling \n':
                text+=lin.lower()
x=sent_tokenize(text)
test_sentences=[]
test_sentencewise_tokens=[]
for l in x:
    li=''.join(l.splitlines())
    test_sentences.append(li)
    test_sentencewise_tokens.append(word_tokenize(li))
test_tokens=word_tokenize(text)



In [181]:
len(test_sentences)

11138

In [182]:
len(test_tokens)

256058

In [183]:
test_final_tokens=[]
for token in test_tokens:
    if not (len(token)==1 and not token.isalnum()):
        test_final_tokens.append(token.strip('.'))
test_tokens=test_final_tokens
#REMOVE PUNCTUATION

In [184]:
len(test_tokens)

204766

In [185]:
test_vocab=set(test_tokens)

In [186]:
len(test_vocab)

11449

In [187]:
vocab=set(vocab)

In [188]:
#OOV words
unknum=len(test_vocab.difference(vocab))
print(unknum)

2068


In [112]:
test_final_tokens=[]
for token in test_tokens:
    if token in vocab:
        test_final_tokens.append(token)

test_tokens=test_final_tokens
#REMOVE PUNCTUATION

In [189]:
test_book=' '.join(test_tokens)

# PERPLEXITY

In [18]:
generate_sentence(2,10)

 the door and the door and the door and the


In [19]:
len(vocab)

21079

In [21]:
def log_perp(n,s):
    words=s.split()
    sum=0
    for i in range(len(words)):
        lis=[]
        for j in range(i,i-n,-1):
            if j>=0:
                lis.append(words[j])
        lis.reverse()
        sz=len(lis)
        if sz==1:
            tup=tuple(lis)
            ratio=ngrams[0][tup]/len(tokens)
            sum+=math.log2(ratio)
        else:
            tup1=tuple(lis)
            lis=lis[:-1]
            tup2=tuple(lis)
            ratio=ngrams[sz-1][tup1]/ngrams[sz-2][tup2]
            sum+=math.log2(ratio)
    return sum*(-1/len(words))

In [22]:
s='the door and the door and the door and the'
print(log_perp(2,s))


4.579554625674844


# SMOOTHING

### ADD-1 

In [190]:
def log_perp1(n,s):
    words=s.split()
    sum=0
    for i in range(len(words)):
        lis=[]
        for j in range(i,i-n,-1):
            if j>=0:
                lis.append(words[j])
        lis.reverse()
        sz=len(lis)
        if sz==1:
            tup=tuple(lis)
            if tup in ngrams[0]:
                ratio=(ngrams[0][tup]+1)/(len(tokens)+len(vocab))
            else:
                # ratio=(1)/(len(tokens)+len(vocab))
                ratio=(1+unknum)/(len(tokens)+len(vocab)+len(test_vocab))
            sum+=math.log2(ratio)
        else:
            tup1=tuple(lis)
            lis=lis[:-1]
            tup2=tuple(lis)
            if tup1 in ngrams[sz-1]:
                ratio=(ngrams[sz-1][tup1]+1)/(ngrams[sz-2][tup2]+len(ngrams[sz-2]))
            elif tup2 in ngrams[sz-2]:
                ratio=(1+unknum)/(ngrams[sz-2][tup2]+len(ngrams[sz-2])+len(test_vocab))
            else:
                ratio=(1+unknum)/(len(ngrams[sz-2])+len(test_vocab))
            sum+=math.log2(ratio)
    return sum*(-1/len(words))

In [191]:
generate_sentence(5,10)

 the door and turned the key fumbling in their panic


In [192]:
#comparison over generated sentences
s='the door and turned the key fumbling in their panic'
n_val=5
print('Log2-perplexity without smoothing is',log_perp(n_val,s))
print('Log2-perplexity with add-1 smoothing is',log_perp1(n_val,s))

Log2-perplexity without smoothing is 1.9812421762636143
Log2-perplexity with add-1 smoothing is 15.077224482717945


In [193]:
s=train_book
for n_val in range(1,6):
    print('Log2-perplexity for n =',n_val,'with add-1 smoothing is',log_perp1(n_val,s))

Log2-perplexity for n = 1 with add-1 smoothing is 9.68405969187683
Log2-perplexity for n = 2 with add-1 smoothing is 10.37694419347486
Log2-perplexity for n = 3 with add-1 smoothing is 16.14611767521765
Log2-perplexity for n = 4 with add-1 smoothing is 17.921149170446203
Log2-perplexity for n = 5 with add-1 smoothing is 18.379349353423052


In [194]:
s=test_book
for n_val in range(1,6):
    print('Log2-perplexity for n =',n_val,'with add-1 smoothing is',log_perp1(n_val,s))

Log2-perplexity for n = 1 with add-1 smoothing is 9.624973962147365
Log2-perplexity for n = 2 with add-1 smoothing is 8.285671163168145
Log2-perplexity for n = 3 with add-1 smoothing is 9.982676532703668
Log2-perplexity for n = 4 with add-1 smoothing is 9.296837041067624
Log2-perplexity for n = 5 with add-1 smoothing is 8.953607757258775


### GOOD TURING

In [195]:
ngrams_total=[] #holds total number of n gram tuples per n
ngrams_freq=[]  #holds dictionary of frequency and count per n
for i in range (n_limit):
    d={}
    ngrams_freq.append(d)
for n in range (n_limit):
    sum=0
    for val in ngrams[n].values():
        # print(val,n)
        if val in ngrams_freq[n]:
            ngrams_freq[n][val]=ngrams_freq[n][val]+1
        else:
            ngrams_freq[n][val]=1
        sum+=val
    ngrams_total.append(sum)

In [198]:
# ngrams_total

In [197]:
# ngrams_freq[4]

In [208]:
ngrams_freq[4][1]

880933

In [249]:
def log_perp2(n,s):
    words=s.split()
    sum=0
    for i in range(len(words)):
        lis=[]
        for j in range(i,i-n,-1):
            if j>=0:
                lis.append(words[j])
        lis.reverse()
        sz=len(lis)
        if sz==1:
            tup=tuple(lis)
            if tup in ngrams[0]:
                freq=ngrams[0][tup]
                if freq+1 in ngrams_freq[0]:
                    ratio=(freq+1)*ngrams_freq[0][freq+1]/(ngrams_freq[0][freq]*ngrams_total[0])
                else:
                    ratio=(freq+1)/(ngrams_freq[0][freq]*ngrams_total[0])
            else:
                if 1 in ngrams_freq[0]:
                    ratio=ngrams_freq[0][1]/len(tokens)
                else :
                    ratio=1/len(tokens)
            sum+=math.log2(ratio)
        else:
            tup1=tuple(lis)
            lis=lis[:-1]
            tup2=tuple(lis)
            if tup1 in ngrams[sz-1]:
                freq=ngrams[sz-1][tup1]
                if freq+1 in ngrams_freq[sz-1]:
                    ratio=(freq+1)*ngrams_freq[sz-1][freq+1]/(ngrams_freq[sz-1][freq]*ngrams_total[sz-1])
                else:
                    ratio=(freq+1)/(ngrams_freq[sz-1][freq]*ngrams_total[sz-1])
            else:
                if 1 in ngrams_freq[sz-1]:
                    #causing low pp
                    ratio=ngrams_freq[sz-1][1]/ngrams_total[sz-1]
                    # ratio=(len(sentences)/len(tokens))*ngrams_freq[sz-1][1]/ngrams_total[sz-1]
                # if unknum+1 in ngrams_freq[sz-1]:
                #     ratio=ngrams_freq[sz-1][1+unknum]/ngrams_total[sz-1]
                else:
                    ratio=1/ngrams_total[sz-1]
            sum+=math.log2(ratio)
    return sum*(-1/len(words))

In [250]:
generate_sentence(5,10)

 the door and turned the key fumbling in their panic


In [251]:
#comparison over generated sentences
s='the door and turned the key fumbling in their panic'
n_val=5
print('Log2-perplexity without smoothing is',log_perp(n_val,s))
print('Log2-perplexity with good turing smoothing is',log_perp2(n_val,s))

Log2-perplexity without smoothing is 1.9812421762636143
Log2-perplexity with good turing smoothing is 18.961096391546445


In [254]:
s=train_book
for n_val in range(1,6):
    print('Log2-perplexity for n =',n_val,'with good turing smoothing is',log_perp2(n_val,s))

Log2-perplexity for n = 1 with good turing smoothing is 9.799544830957888
Log2-perplexity for n = 2 with good turing smoothing is 16.313882079917168
Log2-perplexity for n = 3 with good turing smoothing is 20.19463759448822
Log2-perplexity for n = 4 with good turing smoothing is 22.518577596490413
Log2-perplexity for n = 5 with good turing smoothing is 24.181559692257593


In [255]:
s=test_book
for n_val in range(1,6):
    print('Log2-perplexity for n =',n_val,'with good turing smoothing is',log_perp2(n_val,s))

Log2-perplexity for n = 1 with good turing smoothing is 9.699530164693511
Log2-perplexity for n = 2 with good turing smoothing is 12.02172488585613
Log2-perplexity for n = 3 with good turing smoothing is 6.9359015547532215
Log2-perplexity for n = 4 with good turing smoothing is 2.4575399367718527
Log2-perplexity for n = 5 with good turing smoothing is 0.7209077198811916


### BACKOFF

In [63]:
def log_perp3(n,s):
    words=s.split()
    sum=0
    for i in range(len(words)):
        lis=[]
        for j in range(i,i-n,-1):
            if j>=0:
                lis.append(words[j])
        lis.reverse()
        sz=len(lis)
        # print(sz)
        if sz==1:
            tup=tuple(lis)
            ratio=ngrams[0][tup]/len(tokens)
            sum+=math.log2(ratio)
        else:
            tup1=tuple(lis)
            if tup1 in ngrams[sz-1]:
                
                lis=lis[:-1]
                tup2=tuple(lis)
                ratio=ngrams[sz-1][tup1]/ngrams[sz-2][tup2]
                sum+=math.log2(ratio)
            else :
                lis=lis[1:]
                snew=' '.join(lis)
                back=-1*log_perp3(n-1,snew)*len(lis)
                # print(snew,back)
                sum+=back
                sum+=math.log2(0.4)
                
    return sum*(-1/len(words))

In [64]:
print(log_perp(1,'the'))
print(log_perp(1,'animagi'))
print(log_perp3(2,'animagi the'))
print(log_perp3(1,'the'))

4.4733372813656676
17.00506684057854
11.400166108415785
4.4733372813656676


In [65]:
# log_perp(2,'animagi the')

In [66]:
generate_sentence(5,10)

 the door and turned the key fumbling in their panic


In [67]:
#comparison over generated sentences
s='the door and turned the key fumbling in their panic'
n_val=5
print('Log2-perplexity without smoothing is',log_perp(n_val,s))
print('Log2-perplexity with backoff smoothing is',log_perp3(n_val,s))

Log2-perplexity without smoothing is 1.9812421762636143
Log2-perplexity with backoff smoothing is 1.9812421762636143


In [68]:
#comparison over generated sentences
s='the door and turned the key fumbling in their ministry'
n_val=5
# print('Log2-perplexity without smoothing is',log_perp(n_val,s))
print('Log2-perplexity with backoff smoothing is',log_perp3(n_val,s))

Log2-perplexity with backoff smoothing is 7.710051297553377


### KNESSER-NEY

In [257]:
ngrams_unique_prefix=[]     # for each n gram model contains number of unique w_i-1 for each w_i as key
ngrams_unique_suffix=[]     # for each n gram model contains number of unique w_i for each w_i-1 as key
for i in range (n_limit):
    d={}
    ngrams_unique_prefix.append(d)
    ngrams_unique_suffix.append(d)
for n in range (1,n_limit):
    for key in ngrams[n]:
        lis=list(key)
        lastword=lis[-1]
        lis=tuple(lis[:-1])
        if lastword in ngrams_unique_prefix[n]:
            ngrams_unique_prefix[n][lastword]=ngrams_unique_prefix[n][lastword]+1
        else:
            ngrams_unique_prefix[n][lastword]=1
        if lis in ngrams_unique_suffix[n]:
            ngrams_unique_suffix[n][lis]=ngrams_unique_suffix[n][lis]+1
        else:
            ngrams_unique_suffix[n][lis]=1

In [258]:
def log_perp4(n,s):
    words=s.split()
    sum=0
    for i in range(len(words)):
        lis=[]
        for j in range(i,i-n,-1):
            if j>=0:
                lis.append(words[j])
        lis.reverse()
        sz=len(lis)
        # Absolute discounting
        if sz==1:
            tup=tuple(lis)
            if tup in ngrams[0]:
                ratio=(ngrams[0][tup]-0.75)/len(tokens)
            else :
                ratio = 1/len(tokens)
            sum+=math.log2(ratio)
        else:
            tup1=tuple(lis)
            lis=lis[:-1]
            tup2=tuple(lis)

            # CONTINUATION PROBABILITY
            lastword=lis[-1]
            # pcont_ratio=0
            pcont_ratio=ngrams_unique_prefix[sz-1][lastword]/len(ngrams[sz-1])

            # LAMBDA
            if tup2 in ngrams_unique_suffix[sz-1]:
                lambdamult=0.75*ngrams_unique_suffix[sz-1][tup2]/ngrams[sz-2][tup2]
            else: 
                lambdamult=1
            # lambdamult=1

            if tup1 in ngrams[sz-1]:
                # add-1 smoothing
                # ratio=(max(ngrams[sz-1][tup1]-0.75,0)+1)/(ngrams[sz-2][tup2]+ngrams[sz-2])
                ratio=max(ngrams[sz-1][tup1]-0.75,0)/(ngrams[sz-2][tup2])
                sum+=math.log2(ratio+lambdamult*pcont_ratio)
            else :
                ratio=1/len(ngrams[sz-2])
                sum+=math.log2(ratio+lambdamult*pcont_ratio)

    return sum*(-1/len(words))

In [259]:
generate_sentence(5,10)

 the door and turned the key fumbling in their panic


In [260]:
#comparison over generated sentences
s='the door and turned the key fumbling in their panic'
n_val=5
print('Log2-perplexity without smoothing is',log_perp(n_val,s))
print('Log2-perplexity with KN is',log_perp4(n_val,s))

Log2-perplexity without smoothing is 1.9812421762636143
Log2-perplexity with KN is 2.88767992295894


In [261]:
s=train_book
for n_val in range(1,6):
    print('Log2-perplexity for n =',n_val,'with good turing smoothing is',log_perp4(n_val,s))

Log2-perplexity for n = 1 with good turing smoothing is 9.713219375499346
Log2-perplexity for n = 2 with good turing smoothing is 6.301462416452353
Log2-perplexity for n = 3 with good turing smoothing is 3.9743021816536657
Log2-perplexity for n = 4 with good turing smoothing is 2.6829447301356515
Log2-perplexity for n = 5 with good turing smoothing is 2.291052087376773


In [263]:
s=test_book
for n_val in range(1,6):
    print('Log2-perplexity for n =',n_val,'with good turing smoothing is',log_perp4(n_val,s))

Log2-perplexity for n = 1 with good turing smoothing is 9.837587221150589


KeyError: 'low-'

### INTERPOLATION

In [226]:
import cvxpy as cp
from cvxpy import *
import numpy as np

In [392]:
def log_perp5(n,s):
    words=s.split()
    sum=0
    for i in range(len(words)):
        lis=[]
        for j in range(i,i-n,-1):
            if j>=0:
                lis.append(words[j])
        lis.reverse()
        sz=len(lis)
        
        ratio_lis=[]
        while True:
            tup1=tuple(lis)
            if len(lis)==1:
                ratio=ngrams[0][tup1]/len(tokens)
                ratio_lis.append(ratio)
                break
            elif tup1 in ngrams[len(lis)-1]:
                lis2=lis[:-1]
                tup2=tuple(lis2)
                ratio=ngrams[len(lis)-1][tup1]/ngrams[len(lis)-2][tup2]
                ratio_lis.append(ratio)
            lis=lis[1:]
        ratio_lis.reverse()
        sz=len(ratio_lis)
        final_ratio=0
        for index in range(len(ratio_lis)):
            final_ratio+=ratio_lis[index]*(1/sz)
        sum+=math.log2(final_ratio)
    return sum*(-1/len(words))

In [393]:

def log_perp5conv(n,s):
    optszval=2
    optlis=[]
    for i in range(optszval):
        optlis.append(1)
    A=np.array(optlis)
    words=s.split()
    sum=0
    for i in range(len(words)):
        lis=[]
        for j in range(i,i-n,-1):
            if j>=0:
                lis.append(words[j])
        lis.reverse()
        sz=len(lis)
        
        ratio_lis=[]
        while True:
            tup1=tuple(lis)
            if len(lis)==1:
                ratio=ngrams[0][tup1]/len(tokens)
                ratio_lis.append(ratio)
                break
            elif tup1 in ngrams[len(lis)-1]:
                lis2=lis[:-1]
                tup2=tuple(lis2)
                ratio=ngrams[len(lis)-1][tup1]/ngrams[len(lis)-2][tup2]
                ratio_lis.append(ratio)
            lis=lis[1:]
        ratio_lis.reverse()
        insz=len(ratio_lis)
        while len(ratio_lis)<5:
            ratio_lis.append(1)
        # if insz==5:
        nom=np.array(ratio_lis[:optszval])
        A=np.vstack((A,nom)) 

        sz=len(ratio_lis)
        final_ratio=0
        for index in range(sz):
            final_ratio+=ratio_lis[index]*(1/sz)
        sum+=math.log2(final_ratio)
    # print(A.shape)
    return A

In [321]:
generate_sentence(5,10)

 the door and turned the key fumbling in their panic


In [395]:
s='the door and turned the key fumbling in their panic'
log_perp5(5,s)

2.916425467039723

In [322]:
text=""
filename='Harry_Potter_Text/'+'Book7.txt'
with open(filename) as f:
    Lines = f.readlines()
    # book=""
    for lin in Lines:
        if not (len(lin)>5 and lin[:6]=='Page |'):
            if lin != 'Harry Potter and the Goblet of Fire - J.K. Rowling \n':
                text+=lin.lower()
x=sent_tokenize(text)
test_sentences=[]
test_sentencewise_tokens=[]
for l in x:
    li=''.join(l.splitlines())
    # if not li[-1].isalnum():
    #     li=li[:-1]
    # li= '<s> '+li+' <e>'
    test_sentences.append(li)
    test_sentencewise_tokens.append(word_tokenize(li))
test_tokens=word_tokenize(text)
test_final_tokens=[]
for token in test_tokens:
    if not (len(token)==1 and not token.isalnum()):
        test_final_tokens.append(token.strip('.'))

test_tokens=test_final_tokens
#REMOVE PUNCTUATION

test_final_tokens=[]
for token in test_tokens:
    if token in tokens:
        test_final_tokens.append(token)

test_tokens=test_final_tokens
#REMOVE PUNCTUATION

test_book=' '.join(test_tokens)


#### DEV SET OPTIMIZATION 

In [387]:
s=test_book
n_val=2
A=log_perp5conv(n_val,s)
A = np.delete(A, 0, 0)
print(A.shape)

(200656, 2)


In [372]:
#comparison over generated sentences
# s='the door and turned the key fumbling in their panic'
# n_val=5
# A=log_perp5conv(n_val,s)
# A = np.delete(A, 0, 0)
# print(A.shape)

In [388]:
A[:20]

array([[1.20349896e-02, 1.00000000e+00],
       [4.50185287e-02, 3.60978251e-04],
       [9.07973221e-04, 1.25934861e-02],
       [3.20397249e-04, 1.49521531e-01],
       [1.08609237e-06, 1.00000000e+00],
       [4.50185287e-02, 1.00000000e+00],
       [9.00370575e-04, 1.83353438e-03],
       [4.01854177e-05, 3.61881785e-03],
       [2.59576076e-04, 1.00000000e+00],
       [3.95446232e-03, 5.85774059e-02],
       [1.91619277e-02, 4.34221368e-01],
       [4.23576024e-05, 9.63554951e-04],
       [1.88904046e-02, 2.56410256e-02],
       [6.22330928e-04, 2.67923877e-02],
       [8.68873896e-06, 1.74520070e-03],
       [1.18384068e-04, 1.25000000e-01],
       [1.12975328e-02, 1.83486239e-02],
       [4.50185287e-02, 2.80330706e-01],
       [6.73377269e-05, 3.61881785e-04],
       [6.51655422e-06, 1.00000000e+00]])

In [389]:
x = cp.Variable(2)
objective = cp.Maximize(sum(A @ x ))
constraints = [0<= x, x <= 1, x[0]+x[1] == 1]
prob = cp.Problem(objective, constraints)

In [390]:
result = prob.solve()
print(result)

58419.45500736246


In [391]:
# The optimal value for x is stored in `x.value`.
print(x.value)

[3.89995531e-10 1.00000000e+00]


## PERFORMANCE ON DEV SET

In [38]:
text=""
filename='Harry_Potter_Text/'+'Book7.txt'
with open(filename) as f:
    Lines = f.readlines()
    # book=""
    for lin in Lines:
        if not (len(lin)>5 and lin[:6]=='Page |'):
            if lin != 'Harry Potter and the Goblet of Fire - J.K. Rowling \n':
                text+=lin.lower()
x=sent_tokenize(text)
test_sentences=[]
test_sentencewise_tokens=[]
for l in x:
    li=''.join(l.splitlines())
    # if not li[-1].isalnum():
    #     li=li[:-1]
    # li= '<s> '+li+' <e>'
    test_sentences.append(li)
    test_sentencewise_tokens.append(word_tokenize(li))
test_tokens=word_tokenize(text)





In [39]:
test_final_tokens=[]
for token in test_tokens:
    if not (len(token)==1 and not token.isalnum()):
        test_final_tokens.append(token.strip('.'))

test_tokens=test_final_tokens
#REMOVE PUNCTUATION


In [40]:
len(test_tokens)

204766

In [41]:
test_final_tokens=[]
for token in test_tokens:
    if token in vocab:
        test_final_tokens.append(token)

test_tokens=test_final_tokens
#REMOVE PUNCTUATION

In [42]:
len(test_tokens)

201487

In [104]:
test_book=' '.join(test_tokens)

In [131]:
n_val=2
print('Log2-perplexity by add-1 is',log_perp1(n_val,test_book))
print('Log2-perplexity by GT is',log_perp2(n_val,test_book))
print('Log2-perplexity by backoff is',log_perp3(n_val,test_book))
print('Log2-perplexity by KN is',log_perp4(n_val,test_book))
print('Log2-perplexity by interpolation is',log_perp5(n_val,test_book))

Log2-perplexity by add-1 is 10.823696554299044
Log2-perplexity by GT is 12.286189040700979
Log2-perplexity by backoff is 7.88613436879754
Log2-perplexity by KN is 7.6352386766761144
Log2-perplexity by interpolation is 8.009244242918252


In [136]:
for n in range(1,n_limit+1):
    n_val=n
    print('nval is ',n)
    print('Log2-perplexity by add-1 is',log_perp1(n_val,test_book))
    print('Log2-perplexity by GT is',log_perp2(n_val,test_book))
    print('Log2-perplexity by backoff is',log_perp3(n_val,test_book))
    print('Log2-perplexity by KN is',log_perp4(n_val,test_book))
    print('Log2-perplexity by interpolation is',log_perp5(n_val,test_book))
    # print('')

nval is  1
Log2-perplexity by add-1 is 9.637674198640784
Log2-perplexity by GT is 9.744650881479842
Log2-perplexity by backoff is 9.63596637019401
Log2-perplexity by KN is 9.674584457906617
Log2-perplexity by interpolation is 8.635966370194161
nval is  2
Log2-perplexity by add-1 is 10.823696554299044
Log2-perplexity by GT is 12.286189040700979
Log2-perplexity by backoff is 7.88613436879754
Log2-perplexity by KN is 7.6352386766761144
Log2-perplexity by interpolation is 7.009244242918418
nval is  3
Log2-perplexity by add-1 is 17.070482445307025
Log2-perplexity by GT is 7.093415573536828
Log2-perplexity by backoff is 14.728368738314531
Log2-perplexity by KN is 8.337810310178762
Log2-perplexity by interpolation is 6.513892161228197
nval is  4
Log2-perplexity by add-1 is 19.070664509633033
Log2-perplexity by GT is 2.5042689761335413
Log2-perplexity by backoff is 31.78424852146726
Log2-perplexity by KN is 9.128069518345992
Log2-perplexity by interpolation is 6.401346299165748
nval is  5
Log2